In [3]:
from model import *
import pytchat


C:\Users\avika\OneDrive\Documents\UAL\interactive_dance_thesis


In [4]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="michellejieli/emotion_text_classifier")
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("michellejieli/emotion_text_classifier")
model = AutoModelForSequenceClassification.from_pretrained("michellejieli/emotion_text_classifier")

In [5]:
# Preparing MEED data for model
MEED= prep_MEED_data()
train_data,val_data,frame_dim,max_x,min_x,max_y,min_y,max_dx,min_dx,max_dy,min_dy = MEED

100%|██████████| 4102/4102 [00:03<00:00, 1070.88it/s]


Validating interpolation...
No errors found!


4102it [00:01, 2453.49it/s]
4102it [00:02, 1943.19it/s]


Creating keypoint frames...


100%|██████████| 4102/4102 [00:02<00:00, 1457.60it/s]


...

Validating length of dkp_frames
Validating length of kp_frames
Adding deltas to frames...


4102it [00:02, 1785.03it/s]


Validating length of data after delta
Adding emotions to frames...


100%|██████████| 4102/4102 [00:00<00:00, 37296.20it/s]


Validating length of data after emotions
frame_dim: 107


In [6]:

specific_emotion_video = get_video_by_emotion(train_data, "Anger")

In [7]:
emotion_labels = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']

In [8]:
def visualise_skeleton(all_frames, max_x, max_y, max_frames=500, save=False, save_path=None, prefix=None, train_seed=None , delta=False, destroy = True):
    """Input all frames dim 50xn n being the number of frames 50= 25 keypoints x and y coordinates"""

    
    # visualise to check if the data is correct
    # BODY_25 Keypoints
    keypointsMapping = ['Nose', 'Neck', 'R-Sho', 'R-Elb', 'R-Wr', 'L-Sho', 
                        'L-Elb', 'L-Wr', 'MidHip', 'R-Hip', 'R-Knee', 'R-Ank', 
                        'L-Hip', 'L-Knee', 'L-Ank', 'R-Eye', 'L-Eye', 'R-Ear', 
                        'L-Ear', 'L-BigToe', 'L-SmallToe', 'L-Heel', 'R-BigToe', 
                        'R-SmallToe', 'R-Heel']


    limb_connections = [
        ("Nose", "Neck"),
        ("Neck", "R-Sho"),
        ("R-Sho", "R-Elb"),
        ("R-Elb", "R-Wr"),
        ("Neck", "L-Sho"),
        ("L-Sho", "L-Elb"),
        ("L-Elb", "L-Wr"),
        ("Neck", "MidHip"),
        ("MidHip", "R-Hip"),
        ("R-Hip", "R-Knee"),
        ("R-Knee", "R-Ank"),
        ("MidHip", "L-Hip"),
        ("L-Hip", "L-Knee"),
        ("L-Knee", "L-Ank"),
        ("Nose", "R-Eye"),
        ("R-Eye", "R-Ear"),
        ("Nose", "L-Eye"),
        ("L-Eye", "L-Ear"),
        ("L-Ank", "L-BigToe"),
        ("L-Ank", "L-SmallToe"),
        ("L-Ank", "L-Heel"),
        ("R-Ank", "R-BigToe"),
        ("R-Ank", "R-SmallToe"),
        ("R-Ank", "R-Heel")
    ]
    
     # Define a mapping from emotion vectors to emotion labels
    # Define emotion labels
    emotion_labels = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sad', 'Surprise']
    
    # Initialize a blank canvas (image)
    canvas_size = (int(max_y)+50, int(max_x)+50, 3)  
    canvas = np.zeros(canvas_size, dtype=np.uint8)
    
    
    if save:
        # Determine the save path
        if save_path is None:
            save_path = f"D:\\Interactive Dance Thesis Tests\\TransformerResults\\{train_seed}"

        # Ensure directory exists
        if not os.path.exists(save_path):
            print(f"Creating directory {save_path}")
            os.makedirs(save_path)

        # Determine a unique filename
        existing_files = os.listdir(save_path)
        file_num = 1
        while f"{prefix or ''}{file_num}.mp4" in existing_files:
            file_num += 1
        out_path = os.path.join(save_path, f"{prefix or ''}{file_num}.mp4")

        # Create the video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(out_path, fourcc, 10.0, (canvas_size[1], canvas_size[0]))
    
    previous_frame_data = None
    
    # Iterate over all frames; the first frame uses absolute keypoints, the rest use relative keypoints (deltas)
    for frame_data in tqdm(all_frames[:max_frames], desc="Visualizing frames"):
        
        # If previous_frame_data is None, this is the first frame and we use absolute positions.
        # Otherwise, add the delta to the previous frame's keypoints to get the new keypoints
        if delta ==True:
            if previous_frame_data is not None:
                frame_data[:50] = [prev + delta for prev, delta in zip(previous_frame_data[:50], frame_data[50:100])]
        
            # Update previous_frame_data
            previous_frame_data = copy.deepcopy(frame_data)
        
        canvas_copy = canvas.copy()
        
        # Extract x, y coordinates and emotion vector
        x_coords = frame_data[0:50:2] 
        y_coords = frame_data[1:50:2]
        emotion_vector = tuple(frame_data[100:107])
        
        xy_coords = list(zip(x_coords, y_coords))
        sane = sanity_check(xy_coords)
        # Plot keypoints on the canvas
        for i, (x, y) in enumerate(xy_coords):
            if sane[i] == False:
                continue
            x_val = x.item() if torch.is_tensor(x) else x
            y_val = y.item() if torch.is_tensor(y) else y
            cv2.circle(canvas_copy, (int(x_val), int(y_val)), 3, (0, 0, 255), -1)  
            cv2.putText(canvas_copy, keypointsMapping[i], (int(x_val), int(y_val)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

        # Draw connections (limbs) on the canvas
        for limb in limb_connections:
            start_idx = keypointsMapping.index(limb[0])
            end_idx = keypointsMapping.index(limb[1])
            
            start_point = (int(x_coords[start_idx]), int(y_coords[start_idx]))
            end_point = (int(x_coords[end_idx]), int(y_coords[end_idx]))

            if start_point == (0,0) or end_point == (0,0) or not sane[start_idx] or not sane[end_idx]:
                continue
            cv2.line(canvas_copy, start_point, end_point, (0, 255, 0), 2)  
        
        # Display the emotion percentages and labels on the top right of the frame
        
        emotion_percentages = [f"{int(e * 100)}% {emotion_labels[i]}" for i, e in enumerate(emotion_vector) if round(e * 100) > 1]
        print(emotion_percentages)
        y0, dy = 30, 15  # Starting y position and line gap
        for i, line in enumerate(emotion_percentages):
            y = y0 + i * dy
            cv2.putText(canvas_copy, line, (canvas_size[1] - 120, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)


        # Display the canvas with keypoints and connections
        cv2.imshow("Keypoints Visualization", canvas_copy)

        # If saving, write the frame to the video
        if save:
            out.write(canvas_copy)

        # Wait for 100ms and check for "esc" key press to exit
        key = cv2.waitKey(100)
        if key == 27:  
            break

    # Release the video writer, if used
    if save:
        out.release()

    if destroy:
        # Close the visualization window
        cv2.destroyAllWindows()


In [9]:


# Initialize a dictionary to keep track of cumulative scores and counts for each emotion
emotion_data = {emotion: {"score": 0.0, "count": 0} for emotion in emotion_labels}

# List to store average scores indexed by emotions
average_scores = [0.0 for _ in emotion_labels]
length =0


# create model
CHECKPOINT_PATH = "checkpoints/proto5_checkpoint.pth"

m = MotionModel(input_dim=frame_dim, output_dim=frame_dim, blocksize=BLOCK_SIZE, hidden_dim=512, n_layers=8, dropout=DROPOUT)
m = m.to(device)
optimizer = torch.optim.Adam(m.parameters(), lr=LEARNING_RATE, weight_decay=L2_REG)  # weight_decay=1e-5 L2 regularization
m, optimizer,scheduler, epoch, loss, train_seed = load_checkpoint(m, optimizer, CHECKPOINT_PATH)

# initial generation always neutral
generated = m.generate(specific_emotion_video, FRAMES_GENERATE)
unnorm_out = unnormalise_list_2D(generated, max_x, min_x, max_y, min_y,max_x, min_x, max_y, min_y)

# visualise
visualise_skeleton(unnorm_out[0], max_x, max_y, max_frames=FRAMES_GENERATE,save = False,save_path=None,prefix=f'adam_{EPOCHS}_coord_switchhappytodisgust',train_seed=train_seed,delta=False)


# have to normalise the generated data before putting it into the model

Loading checkpoint...
Checkpoint loaded from checkpoints/proto5_checkpoint.pth


Visualizing frames:   0%|          | 1/300 [00:00<00:40,  7.38it/s]

['100% Anger']
['100% Anger']


Visualizing frames:   1%|          | 3/300 [00:00<00:34,  8.66it/s]

['100% Anger']
['100% Anger']


...

Visualizing frames:  99%|█████████▉| 297/300 [00:32<00:00,  9.17it/s]

['100% Anger']
['100% Anger']


Visualizing frames: 100%|█████████▉| 299/300 [00:32<00:00,  9.21it/s]

['99% Anger']
['101% Anger']


Visualizing frames: 100%|██████████| 300/300 [00:32<00:00,  9.23it/s]


In [10]:
def normalise_generated(unnorm_out, max_x, min_x, max_y, min_y, max_dx, min_dx, max_dy, min_dy):
    norm_out = []
    
    for frame in unnorm_out:
        norm_frame = []
        
        # Normalize the first 50 values (absolute x and y coordinates)
        for i in range(0, 50, 2):
            unnormalized_x = frame[i]
            unnormalized_y = frame[i+1]
            
            norm_x = 2 * (unnormalized_x - min_x) / (max_x - min_x) - 1
            norm_y = 2 * (unnormalized_y - min_y) / (max_y - min_y) - 1
            
            norm_frame.extend([norm_x, norm_y])
        
        # Normalize the second 50 values (x and y deltas)
        for i in range(50, 100, 2):
            unnormalized_dx = frame[i]
            unnormalized_dy = frame[i+1]
            
            norm_dx = 2 * (unnormalized_dx - min_dx) / (max_dx - min_dx) - 1
            norm_dy = 2 * (unnormalized_dy - min_dy) / (max_dy - min_dy) - 1
            
            norm_frame.extend([norm_dx, norm_dy])
        
        # Append the emotion encoding without normalizing
        norm_frame.extend(frame[-7:])
        norm_out.append(norm_frame)
        
    return norm_out


In [11]:
import threading
import queue
import time
import keyboard
import queue


In [12]:

# Initial setup
shared_data = {
    'average_scores': [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
}

chat = pytchat.create(video_id="6v3j-BGsYDg")
FRAMES_GENERATE = 50
terminate_threads = False

# This queue will hold the batches ready for visualization
viz_queue = queue.Queue()


def process_chat_message(c):
    """Process a chat message and update emotion scores."""
    print(f"{c.datetime} [{c.author.name}]- {c.message}")
    result = pipe(c.message)  # Assuming pipe() returns emotion prediction
    print(result)

    detected_emotion = result[0]['label']

    # Reset the counter for the detected emotion and boost its score
    emotion_data[detected_emotion]["count"] = 0
    score = result[0]['score']
    emotion_data[detected_emotion]["score"] = min(1, emotion_data[detected_emotion]["score"] + score)

    # Decay scores for other emotions and increase their counters
    for emotion, data in emotion_data.items():
        if emotion != detected_emotion:
            data["count"] += 1
            if data["count"] >= 5:
                data["score"] = 0
            else:
                data["score"] *= 0.5  # or any other decay factor you prefer

    # Update average scores
    for i, emotion in enumerate(emotion_labels):
        shared_data['average_scores'][i] = emotion_data[emotion]["score"]

    print("Average scores:", shared_data['average_scores'])




# Batch generation function
def generate_new_batch():
    """Generate a new batch based on the current average scores."""
    last_frames = unnorm_out[0][-16:]
    norm_last_frames = normalise_generated(last_frames, max_x, min_x, max_y, min_y, max_x, min_x, max_y, min_y)
    new_batch = [frame[:-7] + shared_data['average_scores'] for frame in norm_last_frames]
    new_input = torch.tensor([new_batch]).to(device).float()

    # Generate the new frames
    generated = m.generate(new_input, FRAMES_GENERATE)
    return unnormalise_list_2D(generated, max_x, min_x, max_y, min_y, max_x, min_x, max_y, min_y)

# Periodic batch generation function - secs
def generate_batches_periodically(period=2):
    """Periodically generate batches and put them in the visualization queue."""
    while True:
        time.sleep(period)
        unnorm_out = generate_new_batch()
        viz_queue.put(unnorm_out)
        
def visualise(unnorm_out):
    # visualize
    visualise_skeleton(unnorm_out[0], max_x, max_y, max_frames=FRAMES_GENERATE,save = False,save_path=None,prefix=f'adam_{EPOCHS}_coord_switchhappytodisgust',train_seed=train_seed,delta=False,destroy=False)

# Visualization thread
def visualise_batches():
    while True:
        # Wait for a new batch to be available in the queue
        unnorm_out = viz_queue.get()
        
        # visualise the batch
        visualise(unnorm_out)

# Start the threads
threading.Thread(target=visualise_batches, daemon=True).start()
threading.Thread(target=generate_batches_periodically, args=(10,), daemon=True).start()

# Process chat messages
length = 0
while chat.is_alive():
    if keyboard.is_pressed('esc'):  # Check if ESC key is pressed
        terminate_threads = True
        break  # Exit the main loop
    for c in chat.get().sync_items():
        process_chat_message(c)
        length += 1
        
cv2.destroyAllWindows()


Visualizing frames:   2%|▏         | 1/50 [00:00<00:06,  7.98it/s]

['100% Anger']
['100% Anger']


Visualizing frames:   6%|▌         | 3/50 [00:00<00:05,  8.90it/s]

['100% Anger']
['100% Anger']


Visualizing frames:  10%|█         | 5/50 [00:00<00:04,  9.18it/s]

...

['99% Happiness']
['98% Happiness']


Visualizing frames:  98%|█████████▊| 49/50 [00:05<00:00,  9.20it/s]

['98% Happiness']
['99% Happiness']


Visualizing frames:   0%|          | 0/50 [00:00<?, ?it/s]

['100% Happiness']
